In [1]:
!pip install git+https://github.com/openai/CLIP.git
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-yw1wzvo6
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-yw1wzvo6
  Resolved https://github.com/openai/CLIP.git to commit a9b1bf5920416aaeaec965c25dd9e8f98c864f16
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 KB 1.2 MB/s eta 0:00:00
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369398 sha256=52a3987b2d42b8071512b14e155ee0df5907a2bcac08c77c46251a8b5716b0f8
  Stored in directory: /tmp/pip-ephem-wheel-cache-fm6dzlqu/wheels/c8/e4/e1/11374c111387672fc2068dfbe0d4b424cb9cdd1b2e184a71b5
Successfully built clip
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 81.0 MB/s eta 0:00:00
     ━━━━━━━━━━

In [2]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [3]:
import clip
import os
from torch import nn
import numpy as np
import torch
import torch.nn.functional as nnf
import sys
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from tqdm import tqdm, trange
from PIL import Image
import pandas as pd
import cv2
import io
import os
import PIL
import random
import numpy as np
import torch
import re
import torchvision
import transformers
# import more_itertools
import matplotlib.pyplot as plt
from torch.utils.data import Dataset
from tqdm import tqdm
from dataclasses import dataclass, field
import torchvision.transforms as T
import torchvision.transforms.functional as TF
import cv2
from PIL import Image
import pickle
from tqdm.contrib import tzip
from tqdm.notebook import tqdm
import gc
import io
import random
import torchvision
from tqdm import tqdm
from dataclasses import dataclass, field
import torchvision.transforms as T
import torchvision.transforms.functional as TF
import cv2
from PIL import Image
import pickle
import sys
from tqdm.contrib import tzip
from tqdm import tqdm
import pandas as pd
import torch
import torch.nn as nn
from torch.nn import functional as nnf
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
import os
import pickle
import sys
import argparse
import json
from typing import Tuple, Optional, Union
from torch.cuda.amp import autocast
from transformers.optimization import Adafactor, AdafactorSchedule
import torch
import io
import os
import PIL
import random
import numpy as np
import torch
import torchvision
import transformers
# import more_itertools
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from torch.utils.data import Dataset
from dataclasses import dataclass, field
import torchvision.transforms as T
import torchvision.transforms.functional as TF
import cv2
from PIL import Image
from torch.utils.checkpoint import checkpoint_sequential

In [ ]:
import tarfile

path = 'train.tar.gz'
tar = tarfile.open(path, "r:gz")
tar.extractall()

In [ ]:
path = '/videos/'

In [ ]:
def get_vid(idx):
    return path + idx + '.mp4'

In [ ]:
df = pd.read_csv('train.csv')
df.drop(['Unnamed: 0'], axis=1, inplace=True)
df.head()

,video_name,question,answer
0,okC18bPTils,все люди в видео мужчины,нет
1,AAQp3iEJxJc,все люди в видео женщины,нет
2,whcxrkF2hws,"спортсмен №3, играющий в помещении",нет
3,LrputIUn4oY,спортсмен в брюках,нет
4,Mzt-E6pxuUI,ребенок в помещении,да


In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
def read_video(path, transform=None, frames_num=16, window=30):
    frames = []
    cap = cv2.VideoCapture(path)
    
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    
    length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    N = length//(frames_num)

    print(N, length, frames_num)

    current_frame = 1
    for i in range(length):

        ret, frame = cap.read(current_frame)
        if ret and i==current_frame and len(frames)<frames_num:
            size = 64, 64
            frame = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            frame.thumbnail(size, Image.ANTIALIAS)
            frames.append(frame)
            current_frame += N
      
    cap.release()
    return frames

In [ ]:
df_train = pd.read_csv('train.csv')

In [ ]:
import os
data = []
for video_name, question, answer in zip(df_train.video_name, df_train.question, df_train.answer):
    name = f'videos/{video_name}.mp4'
    if os.path.exists(name):  
        data += [(name,f'Q: {question} A: {answer}')]

In [ ]:
data[0]

('videos/okC18bPTils.mp4', 'Q: все люди в видео мужчины A: нет')

In [ ]:
def image_grid(imgs, rows, cols):
    pils = imgs
    
    assert len(imgs) == rows*cols

    w, h = imgs[0].size
    grid = Image.new('RGB', size=(cols*w, rows*h))
    grid_w, grid_h = grid.size
    
    for i, img in enumerate(imgs):
        grid.paste(img, box=(i%cols*w, i//cols*h))
        
    return grid

In [ ]:
device = torch.device('cuda')
clip_model_type = "ViT-L/14@336px"

out_path = f"Features_train_full_ru.pkl"
video_path =  'videos'

clip_model, preprocess = clip.load(clip_model_type, device=device, jit=False)
clip_model.to(device)

In [ ]:
all_embeddings = []
all_captions = []
i = 0

for video_name, question, answer in tzip(df_train.video_name, df_train.question, df_train.answer):
    
    
    name = f'{video_path}/{video_name}.mp4'
    
    text = f'Q: {question} A: {answer}'
    #print(name)
    if os.path.exists(name):
        
        video = read_video(path = name, frames_num=9)
        if len(video)>1:
            #print(len(video))
            image = image_grid(video, 3, 3)

            image = preprocess(image).unsqueeze(0).to(device)
            with torch.no_grad():
                prefix = clip_model.encode_image(image).cpu()
            #d["clip_embedding"] = i
            all_embeddings.append(prefix)
            all_captions.append(text)
    
with open(out_path, 'wb') as f:
    pickle.dump({"clip_embedding": torch.cat(all_embeddings, dim=0), "captions": all_captions}, f)

print('Done')
print("%0d embeddings saved " % len(all_embeddings))

In [ ]:
class ClipCocoDataset(Dataset):    
    def __init__(self, data_path: str,  prefix_length= 50, gpt2_type = "sberbank-ai/rugpt3large_based_on_gpt2",
                 normalize_prefix=False):
        self.tokenizer = GPT2Tokenizer.from_pretrained(gpt2_type)
        self.prefix_length = prefix_length
        self.normalize_prefix = normalize_prefix

        with open(data_path, 'rb') as f:
            all_data = pickle.load(f)

        print("Data size is %0d" % len(all_data["clip_embedding"]))

        sys.stdout.flush()
        self.prefixes = all_data["clip_embedding"]
        captions_raw = all_data["captions"]
        
        self.captions = captions_raw   
        self.captions_tokens = []
        self.caption2embedding = []
        max_seq_len = 0        
        i = 0

        for caption in tqdm(captions_raw):
            self.captions_tokens.append(torch.tensor(self.tokenizer.encode(caption), dtype=torch.int64))
            self.caption2embedding.append(self.prefixes[i])
            i += 1
            max_seq_len = max(max_seq_len, self.captions_tokens[-1].shape[0])          
    
        all_len = torch.tensor([len(self.captions_tokens[i]) for i in range(len(self))]).float()

        self.max_seq_len = min(int(all_len.mean() + all_len.std() * 10), int(all_len.max()))

    def pad_tokens(self, item: int):
        tokens = self.captions_tokens[item]
        padding = self.max_seq_len - tokens.shape[0]

        if padding > 0:
            tokens = torch.cat((tokens, torch.zeros(padding, dtype=torch.int64) - 1))
            self.captions_tokens[item] = tokens
        elif padding < 0:
            tokens = tokens[:self.max_seq_len]
            self.captions_tokens[item] = tokens

        mask = tokens.ge(0)  
        tokens[~mask] = 0
        mask = mask.float()
        mask = torch.cat((torch.ones(self.prefix_length), mask), dim=0) 

        return tokens, mask
    
    def __len__(self) -> int:
        return len(self.captions_tokens)   

    def __getitem__(self, item):
        tokens, mask = self.pad_tokens(item)
        prefix = self.prefixes[item]

        if self.normalize_prefix:
            prefix = prefix.float()
            prefix = prefix / prefix.norm(2, -1)

        return tokens, mask, prefix    

In [ ]:
dataset = ClipCocoDataset('Features_train_full_ru.pkl', prefix_length=30, normalize_prefix=False)

Data size is 25228


100%|██████████| 25228/25228 [00:04<00:00, 5314.85it/s]


In [ ]:
# class MappingType(Enum):
#     MLP = 'mlp'
#     Transformer = 'transformer'


# class MlpTransformer(nn.Module):
#      def __init__(self, in_dim, h_dim, out_d: Optional[int] = None, act=nnf.relu, dropout=0.):
#          super().__init__()
#          out_d = out_d if out_d is not None else in_dim
#          self.fc1 = nn.Linear(in_dim, h_dim)
#          self.act = act
#          self.fc2 = nn.Linear(h_dim, out_d)
#          self.dropout = nn.Dropout(dropout)

#      def forward(self, x):
#          x = self.fc1(x)
#          x = self.act(x)
#          x = self.dropout(x)
#          x = self.fc2(x)
#          x = self.dropout(x)
#          return x

# class MLP(nn.Module):

#     def forward(self, x: torch.Tensor) -> torch.Tensor:
#         return self.model(x)

#     def __init__(self, sizes: Tuple[int, ...], bias=True, act=nn.Tanh):
#         super(MLP, self).__init__()
#         layers = []
#         for i in range(len(sizes) - 1):
#             layers.append(nn.Linear(sizes[i], sizes[i + 1], bias=bias))
#             if i < len(sizes) - 2:
#                 layers.append(act())
#         self.model = nn.Sequential(*layers)


# class MultiHeadAttention(nn.Module):

#     def __init__(self, dim_self, dim_ref, num_heads, bias=True, dropout=0.):
#         super().__init__()
#         self.num_heads = num_heads
#         head_dim = dim_self // num_heads
#         self.scale = head_dim ** -0.5
#         self.to_queries = nn.Linear(dim_self, dim_self, bias=bias)
#         self.to_keys_values = nn.Linear(dim_ref, dim_self * 2, bias=bias)
#         self.project = nn.Linear(dim_self, dim_self)
#         self.dropout = nn.Dropout(dropout)

#     def forward(self, x, y=None, mask=None):
#         y = y if y is not None else x
#         b, n, c = x.shape
#         _, m, d = y.shape
#         # b n h dh
#         queries = self.to_queries(x).reshape(b, n, self.num_heads, c // self.num_heads)
#         # b m 2 h dh
#         keys_values = self.to_keys_values(y).reshape(b, m, 2, self.num_heads, c // self.num_heads)
#         keys, values = keys_values[:, :, 0], keys_values[:, :, 1]
#         attention = torch.einsum('bnhd,bmhd->bnmh', queries, keys) * self.scale
#         if mask is not None:
#             if mask.dim() == 2:
#                 mask = mask.unsqueeze(1)
#             attention = attention.masked_fill(mask.unsqueeze(3), float("-inf"))
#         attention = attention.softmax(dim=2)
#         out = torch.einsum('bnmh,bmhd->bnhd', attention, values).reshape(b, n, c)
#         out = self.project(out)
#         return out, attention


# class TransformerLayer(nn.Module):

#     def forward_with_attention(self, x, y=None, mask=None):
#         x_, attention = self.attn(self.norm1(x), y, mask)
#         x = x + x_
#         x = x + self.mlp(self.norm2(x))
#         return x, attention

#     def forward(self, x, y=None, mask=None):
#         x = x + self.attn(self.norm1(x), y, mask)[0]
#         x = x + self.mlp(self.norm2(x))
#         return x

#     def __init__(self, dim_self, dim_ref, num_heads, mlp_ratio=4., bias=False, dropout=0., act=nnf.relu,
#                  norm_layer: nn.Module = nn.LayerNorm):
#         super().__init__()
#         self.norm1 = norm_layer(dim_self)
#         self.attn = MultiHeadAttention(dim_self, dim_ref, num_heads, bias=bias, dropout=dropout)
#         self.norm2 = norm_layer(dim_self)
#         self.mlp = MlpTransformer(dim_self, int(dim_self * mlp_ratio), act=act, dropout=dropout)


# class Transformer(nn.Module):

#     def forward_with_attention(self, x, y=None, mask=None):
#         attentions = []
#         for layer in self.layers:
#             x, att = layer.forward_with_attention(x, y, mask)
#             attentions.append(att)
#         return x, attentions

#     def forward(self, x, y=None, mask=None):
#         for i, layer in enumerate(self.layers):
#             if i % 2 == 0 and self.enc_dec: # cross
#                 x = layer(x, y)
#             elif self.enc_dec:  # self
#                 x = layer(x, x, mask)
#             else:  # self or cross
#                 x = layer(x, y, mask)
#         return x

#     def __init__(self, dim_self: int, num_heads: int, num_layers: int, dim_ref: Optional[int] = None,
#                  mlp_ratio: float = 2., act=nnf.relu, norm_layer: nn.Module = nn.LayerNorm, enc_dec: bool = False):
#         super(Transformer, self).__init__()
#         dim_ref = dim_ref if dim_ref is not None else dim_self
#         self.enc_dec = enc_dec
#         if enc_dec:
#             num_layers = num_layers * 2
#         layers = []
#         for i in range(num_layers):
#             if i % 2 == 0 and enc_dec:  # cross
#                 layers.append(TransformerLayer(dim_self, dim_ref, num_heads, mlp_ratio, act=act, norm_layer=norm_layer))
#             elif enc_dec:  # self
#                 layers.append(TransformerLayer(dim_self, dim_self, num_heads, mlp_ratio, act=act, norm_layer=norm_layer))
#             else:  # self or cross
#                 layers.append(TransformerLayer(dim_self, dim_ref, num_heads, mlp_ratio, act=act, norm_layer=norm_layer))
#         self.layers = nn.ModuleList(layers)


# class TransformerMapper(nn.Module):

#     def forward(self, x):
#         x = self.linear(x).view(x.shape[0], self.clip_length, -1)
#         prefix = self.prefix_const.unsqueeze(0).expand(x.shape[0], *self.prefix_const.shape)
#         prefix = torch.cat((x, prefix), dim=1)
#         out = self.transformer(prefix)[:, self.clip_length:]
#         return out

#     def __init__(self, dim_clip: int, dim_embedding: int, prefix_length: int, clip_length: int, num_layers: int = 8):
#         super(TransformerMapper, self).__init__()
#         self.clip_length = clip_length
#         self.transformer = Transformer(dim_embedding, 8, num_layers)
#         self.linear = nn.Linear(dim_clip, clip_length * dim_embedding)
#         self.prefix_const = nn.Parameter(torch.randn(prefix_length, dim_embedding), requires_grad=True)


# class ClipCaptionModel(nn.Module):

#     def get_dummy_token(self, batch_size: int, device: torch.device) -> torch.Tensor:
#         return torch.zeros(batch_size, self.prefix_length, dtype=torch.int64, device=device)

#     def forward(self, tokens: torch.Tensor, prefix: torch.Tensor, mask: Optional[torch.Tensor] = None,
#                 labels: Optional[torch.Tensor] = None):
#         embedding_text = self.gpt.transformer.wte(tokens)
#         prefix_projections = self.clip_project(prefix).view(-1, self.prefix_length, self.gpt_embedding_size)
#         embedding_cat = torch.cat((prefix_projections, embedding_text), dim=1)
#         if labels is not None:
#             dummy_token = self.get_dummy_token(tokens.shape[0], tokens.device)
#             labels = torch.cat((dummy_token, tokens), dim=1)
#         out = self.gpt(inputs_embeds=embedding_cat, labels=labels, attention_mask=mask)
#         return out

#     def __init__(self, prefix_length: int, clip_length: Optional[int] = None, prefix_size: int = 512,
#                  num_layers: int = 8, mapping_type: MappingType = MappingType.MLP):
#         super(ClipCaptionModel, self).__init__()
#         self.prefix_length = prefix_length
#         self.gpt = GPT2LMHeadModel.from_pretrained('gpt2')
#         self.gpt_embedding_size = self.gpt.transformer.wte.weight.shape[1]
#         if mapping_type == MappingType.MLP:
#             self.clip_project = MLP((prefix_size, (self.gpt_embedding_size * prefix_length) // 2,
#                                      self.gpt_embedding_size * prefix_length))
#         else:
#             self.clip_project = TransformerMapper(prefix_size, self.gpt_embedding_size, prefix_length,
#                                                                      clip_length, num_layers)


# class ClipCaptionPrefix(ClipCaptionModel):

#     def parameters(self, recurse: bool = True):
#         return self.clip_project.parameters()

#     def train(self, mode: bool = True):
#         super(ClipCaptionPrefix, self).train(mode)
#         self.gpt.eval()
#         return self

class MLP(nn.Module):
    def __init__(self, sizes: Tuple[int, ...], bias=True, act=nn.Tanh):
        super(MLP, self).__init__()
        layers = []
        for i in range(len(sizes) - 1):
            layers.append(nn.Linear(sizes[i], sizes[i + 1], bias=bias))
            if i < len(sizes) - 2:
                layers.append(act())
        self.model = nn.Sequential(*layers)
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.model(x)
    
def freeze(
    model,
    freeze_emb=False,
    freeze_ln=False,
    freeze_attn=True,
    freeze_ff=True,
    freeze_other=True,
):
    
    for name, p in model.named_parameters():
    # freeze all parameters except the layernorm and positional embeddings
       
       
        
        name = name.lower()
        if 'ln' in name or 'norm' in name:
            p.requires_grad = not freeze_ln
        elif 'embeddings' in name:
            p.requires_grad = not freeze_emb
        elif 'mlp' in name:
            p.requires_grad = not freeze_ff
        elif 'attn' in name:
            p.requires_grad = not freeze_attn
        else:
            p.requires_grad = not freeze_other
           
    return model

class ClipCaptionModel(nn.Module):
    def __init__(self, backbone, prefix_length: int, prefix_size: int = 768):
          super(ClipCaptionModel, self).__init__()
          self.prefix_length = prefix_length

          self.gpt = GPT2LMHeadModel.from_pretrained(backbone)
          #self.gpt = freeze(self.gpt)
          self.gpt_embedding_size = self.gpt.transformer.wte.weight.shape[1]
          self.clip_project = MLP((prefix_size, (self.gpt_embedding_size * prefix_length) // 2,
                                  self.gpt_embedding_size * prefix_length))

    def get_dummy_token(self, batch_size: int, device: torch.device) -> torch.Tensor:
        return torch.zeros(batch_size, self.prefix_length, dtype=torch.int64, device=device)
    
    def forward(self, tokens: torch.Tensor, prefix: torch.Tensor, mask: Optional[torch.Tensor] = None,
                labels: Optional[torch.Tensor] = None):

        embedding_text = self.gpt.transformer.wte(tokens)
        prefix_projections = self.clip_project(prefix).view(-1, self.prefix_length, self.gpt_embedding_size)
        embedding_cat = torch.cat((prefix_projections, embedding_text), dim=1)

        if labels is not None:
            dummy_token = self.get_dummy_token(tokens.shape[0], tokens.device)
            labels = torch.cat((dummy_token, tokens), dim=1)

        out = self.gpt(inputs_embeds=embedding_cat, labels=labels, attention_mask=mask)

        return out

class ClipCaptionPrefix(ClipCaptionModel):
    def parameters(self, recurse: bool = True):
        return self.clip_project.parameters()

    def train(self, mode: bool = True):
        super(ClipCaptionPrefix, self).train(mode)
        self.gpt.eval()

        return self

def train(dataset, model: ClipCaptionModel, args,
          warmup_steps: int = 5000, output_dir: str = ".", output_prefix: str = ""):
  
    device = torch.device('cuda')
    batch_size = args.bs
    epochs = args.epochs

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    model = model.to(device)
    
    model.train()
    optimizer = AdamW(model.parameters(), lr=args.lr,betas=(0.9, 0.995))

    train_dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, drop_last=True)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=warmup_steps, num_training_steps=epochs * len(train_dataloader)
    )

    for epoch in range(epochs):
        print(f">>> Training epoch {epoch}")
        sys.stdout.flush()
        progress = tqdm(total=len(train_dataloader), desc=output_prefix)
        step = 0

        for idx, (tokens, mask, prefix) in enumerate(train_dataloader):
            model.zero_grad()
            step+=1
            tokens, mask, prefix = tokens.to(device), mask.to(device), prefix.to(device, dtype=torch.float32)
            
            outputs = model(tokens, prefix, mask)
            logits = outputs.logits[:, dataset.prefix_length - 1: -1]

            loss = nnf.cross_entropy(logits.reshape(-1, logits.shape[-1]), tokens.flatten(), ignore_index=0)
            segments = 2

            loss.backward()
            
            optimizer.step()
            scheduler.step()

            progress.set_postfix({"loss": loss.item()})

            optimizer.step()
            scheduler.step()
                        
            progress.update()            

            del tokens
            del mask
            del prefix
            torch.clear_autocast_cache()
            torch.cuda.empty_cache()
            
            if (idx + 1) % 7000 == 0:
                torch.save(
                    model.state_dict(),
                    
                    os.path.join(output_dir, f"{output_prefix}_latest.pt"),
                )

        progress.close()
        if epoch % args.save_every ==0:
            torch.save(
                model.state_dict(),
                os.path.join(output_dir, f"{output_prefix}-{epoch:03d}.pt"),
            )

    return model

class Args():
    def __init__(self):
        self.backbone = 'sberbank-ai/rugpt3small_based_on_gpt2'
        self.data = 'Features_train_full_ru.pkl'
        self.out_dir = 'checkpoints_larger'
        self.prefix = 'prefix_1'
        self.epochs = 10
        self.save_every = 1
        self.prefix_length = 30
        self.bs = 10
        self.only_prefix = False
        self.lr = 2e-5       

def main():    
    args = Args()

    prefix_length = args.prefix_length
    dataset = ClipCocoDataset(args.data, prefix_length)
    
    model = ClipCaptionModel(backbone = args.backbone, prefix_length = prefix_length)
    
    print("Train both prefix and GPT")
    sys.stdout.flush()
    train(dataset, model, args, output_dir=args.out_dir, output_prefix=args.prefix)

In [ ]:
main()

Data size is 25228





  0%|          | 0/25228 [00:00<?, ?it/s]


  2%|▏         | 537/25228 [00:00<00:04, 5367.74it/s]


  4%|▍         | 1074/25228 [00:00<00:04, 5318.52it/s]


  7%|▋         | 1730/25228 [00:00<00:03, 5880.63it/s]


  9%|▉         | 2319/25228 [00:00<00:04, 5680.06it/s]


 12%|█▏        | 3008/25228 [00:00<00:03, 6103.61it/s]


 14%|█▍        | 3621/25228 [00:00<00:04, 5087.62it/s]


 17%|█▋        | 4165/25228 [00:00<00:04, 5187.49it/s]


 19%|█▉        | 4737/25228 [00:00<00:03, 5340.29it/s]


 21%|██        | 5294/25228 [00:00<00:03, 5405.99it/s]


 24%|██▎       | 5979/25228 [00:01<00:03, 5829.02it/s]


 26%|██▌       | 6571/25228 [00:01<00:03, 4840.04it/s]


 28%|██▊       | 7133/25228 [00:01<00:03, 5041.25it/s]


 30%|███       | 7664/25228 [00:01<00:03, 4781.02it/s]


 33%|███▎      | 8257/25228 [00:01<00:03, 5084.06it/s]


 35%|███▌      | 8890/25228 [00:01<00:03, 5425.83it/s]


 37%|███▋      | 9452/25228 [00:01<00:02, 5479.75it/s]


 40%|████      | 10129/25228 [00:01<00:02,

Train both prefix and GPT
>>> Training epoch 0


prefix_1: 100%|██████████| 2522/2522 [11:30<00:00,  3.65it/s, loss=1.17] 


>>> Training epoch 1


...

prefix_1: 100%|██████████| 2522/2522 [11:29<00:00,  3.66it/s, loss=0.394]


>>> Training epoch 8


prefix_1: 100%|██████████| 2522/2522 [11:30<00:00,  3.65it/s, loss=0.463]


>>> Training epoch 9


prefix_1: 100%|██████████| 2522/2522 [11:31<00:00,  3.65it/s, loss=0.657]


In [ ]:
!cp /content/checkpoints/prefix_1-007.pt /content/drive/MyDrive/4

In [ ]:
!cp /content/drive/MyDrive/4/prefix_1-007.pt .

In [ ]:
device = 'cuda'
clip_model, preprocess = clip.load("ViT-L/14@336px", device=device, jit=False)
tokenizer = GPT2Tokenizer.from_pretrained('sberbank-ai/rugpt3large_based_on_gpt2')

prefix_length= 50
model_path = 'checkpoints_larger/prefix_1-004.pt'
model = ClipCaptionModel(backbone = 'sberbank-ai/rugpt3small_based_on_gpt2', prefix_length = 50)

model.load_state_dict(torch.load(model_path, map_location='cpu')) 
model.to(device)
None